In [15]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from sklearn.datasets import load_files
from keras.utils import to_categorical
import cv2
import scipy

In [2]:
train_path = 'data/train'
valid_path = 'data/valid'
test_path = 'data/test'

In [3]:
# loading images for paths
def load_images(path):
    i = load_files(path)
    return i['filenames'],i['target']

In [4]:
train_images,train_targets = load_images(train_path)
valid_images,valid_targets = load_images(valid_path)
test_images,test_targets = load_images(test_path)

In [5]:
# converting raw images to tensors
def split(raw_image):
    img = tf.contrib.keras.preprocessing.image.load_img(raw_image)
    img = tf.contrib.keras.preprocessing.image.img_to_array(img)
    img = cv2.resize(img,(256,256))/255.
    return img
def image_to_tensors(raw_images):
    a = list()
    for i in range(len(raw_images)):
        img = split(raw_images[i])
        a.append(img)
    a = np.array(a)
    return a

In [6]:
train_tensors = image_to_tensors(train_images)
valid_tensors = image_to_tensors(valid_images)
test_tensors = image_to_tensors(test_images)

In [29]:
def one_hot(targets):
    targets = to_categorical(targets)
    return targets

In [30]:
train_targets = one_hot(train_targets)
valid_targets = one_hot(valid_targets)
test_targets = one_hot(test_targets)

In [34]:
train_targets[0:5]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [40]:
def generate_filters(kernel_size,in_depth,out_depth):
    return tf.Variable(tf.random_uniform([kernel_size,kernel_size,in_depth,out_depth],minval=0.25,maxval=0.75))
def generate_biases(out_depth):
    return tf.Variable(tf.random_uniform([out_depth],minval=0.25,maxval=0.75))
def generate_weights_for_fc(in_nodes,out_nodes):
    return tf.Variable(tf.random_uniform([in_nodes,out_nodes],minval=0.25,maxval=0.75))


"\nweights = {\n    'wc1': tf.Variable(tf.random_uniform([3,3,3,16],minval=0.25,maxval=0.75)),\n    'wc2': tf.Variable(tf.random_uniform([3,3,16,32],minval=0.25,maxval=0.75)),\n    'wc3': tf.Variable(tf.random_uniform([3,3,32,64],minval=0.25,maxval=0.75)),\n    'wc4': tf.Variable(tf.random_uniform([3,3,64,128],minval=0.25,maxval=0.75)),\n    'wc5': tf.Variable(tf.random_uniform([3,3,128,256],minval=0.25,maxval=0.75)),\n    'fc1': tf.Variable(tf.random_uniform([16*16*256,1024],minval=0.25,maxval=0.75)),\n    'fc2': tf.Variable(tf.random_uniform([1024,3],minval=0.25,maxval=0.75))\n}\nbiases = {\n    'bc1': tf.Variable(tf.random_uniform([16],minval=0.25,maxval=0.75)),\n    'bc2': tf.Variable(tf.random_uniform([32],minval=0.25,maxval=0.75)),\n    'bc3': tf.Variable(tf.random_uniform([64],minval=0.25,maxval=0.75)),\n    'bc4': tf.Variable(tf.random_uniform([128],minval=0.25,maxval=0.75)),\n    'bc5': tf.Variable(tf.random_uniform([256],minval=0.25,maxval=0.75)),\n    'bf1': tf.Variable(tf.r

In [61]:
learning_rate = 0.1
epochs = 50
batch_size = 100
dropout = 0.75

In [62]:
def create_conv2d(image_batch,in_depth,out_depth):
    kernel_size = 3
    layer = tf.nn.conv2d(input=image_batch,filter=generate_filters(kernel_size,in_depth,out_depth),strides=[1,1,1,1],padding='SAME')
    layer = tf.nn.bias_add(value=layer,bias=generate_biases(out_depth))
    layer = tf.nn.relu(layer)
    return layer
def create_maxpool(conv_layer):
    layer = tf.nn.max_pool(conv_layer,[1,3,3,1],[1,2,2,1],padding='SAME')
    return layer
def create_fc_after_conv(conv_layer,in_nodes,out_nodes,keep_prob):
    layer = tf.reshape(conv_layer,[-1,16*16*256])
    layer = tf.add(tf.matmul(layer,generate_weights_for_fc(in_nodes,out_nodes)),generate_biases(out_nodes))
    layer = tf.nn.relu(layer)
    layer = tf.nn.dropout(layer,keep_prob)
    return layer
def create_fc_after_fc(fc,in_nodes,out_nodes,keep_prob):
    layer = tf.add(tf.matmul(fc,generate_weights_for_fc(in_nodes,out_nodes)),generate_biases(out_nodes))
    layer = tf.nn.relu(layer)
    layer = tf.nn.dropout(layer,keep_prob)
    return layer
def next_batch(image_batch,train_targets,batch,batch_size):
    start = batch*batch_size
    end = start + batch_size
    min_train_batch = image_batch[start:end]
    min_targets_batch = train_targets[start:end]
    train_datagen = ImageDataGenerator(rotation_range=60,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,
                                  shear_range=20,zoom_range=[0.2,0.2])
    train_datagen.fit(x=min_train_batch,augment=True,rounds=2)
    t = 0
    for x_batch,y_batch in train_datagen.flow(min_train_batch,min_targets_batch,batch_size=batch_size):
        t += 1
        if t > 1:
            break
        return x_batch,y_batch
        


In [63]:
def convnet(x,keep_prob):
    conv1 = create_conv2d(x,3,16)
    max1 = create_maxpool(conv1)
    conv2 = create_conv2d(max1,16,32)
    max2 = create_maxpool(conv2)
    conv3 = create_conv2d(max2,32,64)
    max3 = create_maxpool(conv3)
    conv4 = create_conv2d(max3,64,128)
    max4 = create_maxpool(conv4)
    conv5 = create_conv2d(max4,128,256)
    fc1 = create_fc_after_conv(conv5,16*16*256,1024,dropout)
    fc2 = create_fc_after_fc(fc1,1024,2048,dropout)
    fc3 = tf.add(tf.matmul(fc2,generate_weights_for_fc(2048,3)),generate_biases(3))
    fc3 = tf.nn.softmax(fc3)
    return fc3

In [ ]:
x = tf.placeholder(tf.float32,[None,256,256,3])
y = tf.placeholder(tf.float32,[None,3])
keep_prob = tf.placeholder(tf.float32)

logits = convnet(x,keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))

accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init = tf.global_variables_initializer()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type ='BFC'
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(init)
    for epoch in range(epochs):
        for batch in range(train_tensors.shape[0]//batch_size):
            batch_x,batch_y = next_batch(train_tensors,train_targets,batch,batch_size)
            #batch_y = next_batch(train_targets,batch_size)
            
            sess.run(optimizer,feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob:dropout
            })
            loss = sess.run(cost,feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.
            })
            valid_acc = sess.run(accuracy,feed_dict={
                x:valid_tensors[:],
                y:valid_targets[:],
                keep_prob:1.
            })
            
            print('Epoch {:>2}, Batch{:>3},loss {:>3},Validation_accuracy{:>3}'.format(epoch+1,batch+1,loss,valid_acc))
            

Epoch  1, Batch  1,loss 1.3914446830749512,Validation_accuracy0.2866666615009308
Epoch  1, Batch  2,loss 1.3614447116851807,Validation_accuracy0.273333340883255
Epoch  1, Batch  3,loss 1.4114446640014648,Validation_accuracy0.2666666805744171
Epoch  1, Batch  4,loss 1.4514447450637817,Validation_accuracy0.2933333218097687
Epoch  1, Batch  5,loss 1.4714447259902954,Validation_accuracy0.2666666805744171
Epoch  1, Batch  6,loss 1.3714447021484375,Validation_accuracy0.2866666615009308
Epoch  1, Batch  7,loss 1.3714447021484375,Validation_accuracy0.2866666615009308
Epoch  1, Batch  8,loss 1.441444754600525,Validation_accuracy0.2866666615009308
Epoch  1, Batch  9,loss 1.2914446592330933,Validation_accuracy0.30000001192092896
Epoch  1, Batch 10,loss 1.441444754600525,Validation_accuracy0.2666666805744171
Epoch  1, Batch 11,loss 1.3814446926116943,Validation_accuracy0.2800000011920929
Epoch  1, Batch 12,loss 1.4214446544647217,Validation_accuracy0.273333340883255
Epoch  1, Batch 13,loss 1.36144

Epoch  6, Batch  3,loss 1.4114446640014648,Validation_accuracy0.2800000011920929
Epoch  6, Batch  4,loss 1.4314446449279785,Validation_accuracy0.2800000011920929
Epoch  6, Batch  5,loss 1.4114446640014648,Validation_accuracy0.2933333218097687
Epoch  6, Batch  6,loss 1.401444673538208,Validation_accuracy0.2866666615009308
Epoch  6, Batch  7,loss 1.3814446926116943,Validation_accuracy0.2933333218097687
Epoch  6, Batch  8,loss 1.441444754600525,Validation_accuracy0.2933333218097687
Epoch  6, Batch  9,loss 1.3114447593688965,Validation_accuracy0.2866666615009308
Epoch  6, Batch 10,loss 1.4214446544647217,Validation_accuracy0.273333340883255
Epoch  6, Batch 11,loss 1.4214446544647217,Validation_accuracy0.30000001192092896
Epoch  6, Batch 12,loss 1.4514447450637817,Validation_accuracy0.2866666615009308
Epoch  6, Batch 13,loss 1.3514447212219238,Validation_accuracy0.2800000011920929
Epoch  6, Batch 14,loss 1.3814446926116943,Validation_accuracy0.2866666615009308
Epoch  6, Batch 15,loss 1.4414

In [26]:
test_acc = sess.run(accuracy,feed_dict={
    x:test_tensors[:],
    y:test_targets[:],
    keep_prob:1.
})
print('Testing accuracy {}'.format(test_acc))

RuntimeError: Attempted to use a closed Session.